In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Activation,Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
traindf = pd.read_csv("../input/fast-furious-and-insured/Fast_Furious_Insured/train.csv", dtype=str)
testdf = pd.read_csv("../input/fast-furious-and-insured/Fast_Furious_Insured/test.csv", dtype = str)

In [ ]:
traindf.head()

In [ ]:
testdf.head()

In [ ]:
from sklearn.model_selection import train_test_split
train,  test = train_test_split(traindf, test_size = 0.15, random_state = 42)

print(f"train set shape: {train.shape}")

print(f"validation set shape: {test.shape}")

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
train_gen = train_datagen.flow_from_dataframe(dataframe = train,
                                              directory = "../input/fast-furious-and-insured/Fast_Furious_Insured/trainImages",
                                              x_col = 'Image_path', y_col ='Condition',
                                              target_size = (200,200), batch_size = 256, 
                                              class_mode = 'binary', shuffle = True)
val_gen = train_datagen.flow_from_dataframe(dataframe = test,
                                            directory = "../input/fast-furious-and-insured/Fast_Furious_Insured/trainImages",
                                            target_size=(200,200), x_col = 'Image_path', y_col ='Condition', 
                                            class_mode='binary',
                                            batch_size= 256, shuffle=True)

test_dir = "../input/fast-furious-and-insured/Fast_Furious_Insured/testImages" 
test_gen = test_datagen.flow_from_dataframe(directory = test_dir, dataframe = testdf, target_size = (200,200),
                                            x_col = 'Image_path', y_col = None, shuffle = False,
                                             batch_size = 256, class_mode = None)

In [ ]:
import tensorflow as tf

base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape = (200,200,3),
                                                     include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()   

In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(10))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_gen,
    validation_data = val_gen,
    epochs = 10)

In [ ]:
from sklearn.metrics import  accuracy_score
#using model.predict as per keras rule to use >0.5 for binary classification
predictions = (model.predict(val_gen)>0.5).astype("int32")

y_test2 = val_gen.labels
print('Accuracy score:\n', accuracy_score(y_test2, predictions))

In [ ]:
model.evaluate(train_gen)

In [ ]:
model.evaluate(val_gen)

In [ ]:
preds = (model.predict(test_gen)>0.5).astype("int32")

preds

In [ ]:
testdf['Condition'] = pd.DataFrame(preds)

In [ ]:
testdf.head()

In [ ]:
testdf.Condition.value_counts()

In [ ]:
traindf.Condition.value_counts()

### we have classified images where 0 : Not Damaged and 1: Damaged
### we don't need images now to predict the amount
### So we can drop the images
### we train them on the train set and predict on the test set

In [ ]:
## we have classified images where 0 : Not Damaged and 1: Damaged
## we don't need images now to predict the amount
## So we drop the images
## we train them on the train set and predict on the test set

traindf.head()

In [ ]:
#handling datetime

#train_data
traindf.Expiry_date = traindf.Expiry_date.apply(pd.to_datetime)
traindf['month'] = traindf.Expiry_date.apply(lambda x: x.month)
traindf['day'] = traindf.Expiry_date.apply(lambda x: x.day)
traindf['year'] = traindf.Expiry_date.apply(lambda x: x.year)
traindf.drop(['Expiry_date'], 1, inplace = True)

#test_data
testdf.Expiry_date = testdf.Expiry_date.apply(pd.to_datetime)
testdf['month'] = testdf.Expiry_date.apply(lambda x: x.month)
testdf['day'] = testdf.Expiry_date.apply(lambda x: x.day)
testdf['year'] = testdf.Expiry_date.apply(lambda x: x.year)
testdf.drop(['Expiry_date'], 1, inplace = True)


In [ ]:
traindf.Insurance_company.value_counts()

In [ ]:
testdf.Insurance_company.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()   

#train
traindf['Insurance_company'] = le.fit_transform(traindf['Insurance_company'])
#test
testdf['Insurance_company'] = le.fit_transform(testdf['Insurance_company'])

In [ ]:
traindf = traindf.drop(['Image_path'], axis = 1)
testdf= testdf.drop(['Image_path'], axis = 1)

In [ ]:
traindf.head()

In [ ]:
testdf.head()

In [ ]:
traindf = traindf.astype(float)
testdf = testdf.astype(float)

In [ ]:
traindf.info()

In [ ]:
traindf2 = traindf.dropna()
testdf2 = testdf.dropna()

In [ ]:
traindf2.corr().abs()['Amount'].sort_values(ascending = False)

In [ ]:
testdf2.info()

In [ ]:
testdf2 = testdf2.drop([ 'day', 'month', 'year'], axis = 1)

In [ ]:
X = traindf2.drop(['Amount', 'day', 'month', 'year'], axis = 1)
y =traindf2['Amount']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(n_estimators = 1000, max_depth=10)
model2.fit(X_train, y_train)
pred_new = model2.predict(X_test)


In [ ]:
test_pred = model2.predict(testdf2)

In [ ]:
print(test_pred)

In [ ]:
test_df = pd.read_csv("../input/fast-furious-and-insured/Fast_Furious_Insured/test.csv")

In [ ]:
submission = pd.DataFrame({'Image_path': test_df.Image_path, 'Condition': testdf.Condition, 
                          'Amount': test_pred})
# you could use any filename. We choose submission here
submission.to_csv('submission.csv', index=False)

# Upvote if you like it or fork it :)